## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2024xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [2]:
from datetime import datetime
import json
from tab_dataset.cdataset import Cdataset
import pandas as pd
import ntv_pandas as npd
import pathlib

def analyse_integrite(data, fields, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le dict 'fields'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement ('indic') affiché. 
    La liste des lignes en erreur est optionnellement ajoutée ('affiche') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(fields)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [3]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23', '2023-12-14', '2023-12-29',
         '2024-01-15', '2024-02-07', '2024-02-28', '2024-03-27']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.3.1-'+date[:4]+date[5:7]+date[8:]+'.csv', # après 2024-02-06
    #'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
    'chemin': str(pathlib.Path(npd.__file__).parent.parent.parent/"Environmental-Sensing"/"python"/"Validation"/"irve"/"Analyse")
    # 'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/gen-ntv/python/Validation/irve/Analyse/'}
    # log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
    #'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
    #'chemin': 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
      }
irve = pd.read_csv(log['chemin'] + '/' + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  87416


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [4]:
# complément à inclure dans le schéma de données
fields = [
 # relation unicité des pdl
     { "name": "index",
       "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 4 % des points de charge sont hors itinerance

In [5]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'], 'soit : ', len(non_concerne)/(len(non_concerne)+len(itinerance)))
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  907 soit :  0.01037567493365059
nombre de pdc en itinerance   :  86509


In [6]:
# arborescence des champs et nombre de valeurs différentes
print(itinerance_init.npd.analysis().tree())

-1: root-derived (86509)
   0 : index (0 - 86509)
   1 : contact_operateur (86344 - 165)
   2 : nom_enseigne (83444 - 3065)
   3 : coordonneesXY (61922 - 24587)
   4 : adresse_station (62168 - 24341)
   5 : id_station_itineranc (51415 - 35094)
   6 : nom_station (61400 - 25109)
   7 : implantation_station (86504 - 5)
   8 : nbre_pdc (86446 - 63)
   9 : condition_acces (86507 - 2)
   10: horaires (85915 - 594)
   11: station_deux_roues (86503 - 6)
   12: id_pdc_itinerance (8080 - 78429)
   13: date_maj (85815 - 694)
   14: last_modified (86009 - 500)


-----------------------
## 4 - Bilan initial intégrité
- 25 % des lignes présentent un défaut d'intégrité

In [7]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          16158
contact_operateur - id_station_itinerance           9938
nom_enseigne - id_station_itinerance               10551
coordonneesXY - id_station_itinerance              11327
id_station_itinerance - id_pdc_itinerance           5738
nom_station - id_station_itinerance                10713
implantation_station - id_station_itinerance        4863
nbre_pdc - id_station_itinerance                    7755
condition_acces - id_station_itinerance               83
horaires - id_station_itinerance                      80
station_deux_roues - id_station_itinerance         10478
adresse_station - coordonneesXY                     1811

nombre d'enregistrements sans erreurs :  64954
nombre d'enregistrements avec au moins une erreur :  21555
taux d'erreur :  25  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 35% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 50% le nombre de lignes erronnées

In [8]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'], ' soit ', round(log['sans_doublons_pdc']/log['init_ko'] * 100), ' %')

nombre de doublons pdc :  8080  soit  37  %
nombre de pdc sans doublon   :  13475  soit  63  %


In [9]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance            556
nom_enseigne - id_station_itinerance                 559
coordonneesXY - id_station_itinerance               1330
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 1316
implantation_station - id_station_itinerance         511
nbre_pdc - id_station_itinerance                    3493
condition_acces - id_station_itinerance               31
horaires - id_station_itinerance                      53
station_deux_roues - id_station_itinerance          1117
adresse_station - coordonneesXY                     1348

nombre d'enregistrements sans erreurs :  7558
nombre d'enregistrements avec au moins une erreur :  5917


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 25% supplémentaire le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 43 opérateurs et sont liées à des causes multiples

In [10]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  790  soit  4  %
nombre de pdc sans doublon   :  5127


In [11]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                   5
coordonneesXY - id_station_itinerance                332
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  410
implantation_station - id_station_itinerance          21
nbre_pdc - id_station_itinerance                    2982
condition_acces - id_station_itinerance                4
horaires - id_station_itinerance                      29
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     1304

nombre d'enregistrements sans erreurs :  304
nombre d'enregistrements avec au moins une erreur :  4823  soit  82  %


In [12]:
# structure des données présentant une erreur et nombre de valeurs
print(itinerance_5.npd.analysis().tree())

-1: root-derived (4823)
   0 : index (0 - 4823)
   1 : contact_operateur (4779 - 44)
   2 : nom_enseigne (4673 - 150)
   3 : coordonneesXY (4115 - 708)
   4 : adresse_station (3931 - 892)
   5 : id_station_itineranc (3356 - 1467)
      13: date_maj (1378 - 89)
   6 : nom_station (3986 - 837)
   7 : implantation_station (4818 - 5)
   8 : nbre_pdc (4804 - 19)
   9 : condition_acces (4821 - 2)
   10: horaires (4774 - 49)
   11: station_deux_roues (4818 - 5)
   12: id_pdc_itinerance (0 - 4823)
   14: last_modified (4747 - 76)


In [13]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  8


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [14]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  4823
total des doublons à supprimer :  8870
nombre de pdc avec controles ok :  72816


In [15]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [16]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [17]:
# structure des données bonnes
#print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())
print(itinerance_ok.loc[:, mandatory].npd.analysis().tree())

-1: root-derived (72816)
   4 : id_station_itineranc (40060 - 32756)
      1 : nom_enseigne (29821 - 2935)
      2 : coordonneesXY (11726 - 21030)
         3 : adresse_station (404 - 20626)
      5 : nom_station (11409 - 21347)
         0 : contact_operateur (21196 - 151)
      6 : implantation_station (32751 - 5)
      7 : nbre_pdc (32693 - 63)
      8 : condition_acces (32754 - 2)
      9 : horaires (32205 - 551)
      10: station_deux_roues (32750 - 6)
      19: paiement_autre (32747 - 9)
   11: id_pdc_itinerance (0 - 72816)
   12: puissance_nominale (72720 - 96)
   13: prise_type_ef (72808 - 8)
   14: prise_type_2 (72809 - 7)
   15: prise_type_combo_ccs (72809 - 7)
   16: prise_type_chademo (72809 - 7)
   17: prise_type_autre (72809 - 7)
   18: paiement_acte (72808 - 8)
   20: reservation (72809 - 7)
   21: accessibilite_pmr (72812 - 4)
   22: restriction_gabarit (72713 - 103)
   23: date_maj (72264 - 552)
   24: last_modified (72383 - 433)


### Indicateurs

In [18]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [19]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  83  % des écarts

hello@powerdot.fr 2551
supervision-ev.france@totalenergies.com 554
info@ionity.eu 431
sav@izivia.com 287
support@alizecharge.fr 200


In [20]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           3732
77  %


In [21]:
# stockage des indicateurs
log |= indic
log_file = log['chemin'] + '/' + 'logfile.txt'
#with open('logfile.txt', 'a', encoding="utf-8") as f:
with open(log_file, 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (3) (109 pdc)
- exemple station 'FR55CP92140' avec 29 pdc ! : 5 coordonnées XY, 5 nom_station, 2 nbre_pdc)

In [22]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4367,68543,support@alizecharge.fr,Reveo,"[3.083424, 42.526063]",COLLIOURE - Place Du 8 Mai 1945 B01,FRS66P66053001,COLLIOURE - Place Du 8 Mai 1945 B01,Voirie,2,Accès libre,Mo-Su 00:00-23:59,False,FRS66E660530011,2024-02-15,2024-03-07T15:18:29.188000+00:00
4368,68544,support@alizecharge.fr,Reveo,"[3.083424, 42.526063]",COLLIOURE - Place Du 8 Mai 1945 B01,FRS66P66053001,COLLIOURE - Place Du 8 Mai 1945 B01,Voirie,2,Accès libre,Mo-Su 00:00-23:59,False,FRS66E660530012,2024-02-15,2024-03-07T15:18:29.188000+00:00
4369,68545,support@alizecharge.fr,Reveo,"[3.083424, 42.526063]",Place Du 8 Mai 1945 66190 COLLIOURE,FRS66P66053001,COLLIOURE - Place Du 8 Mai 1945 B02,Station dédiée à la recharge rapide,1,Accès libre,Mo-Su 00:00-23:59,False,FRS66E660530013,2024-02-15,2024-03-07T15:18:29.188000+00:00
4576,79972,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550401,2024-02-15,2024-03-07T15:18:29.188000+00:00
4577,79973,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550402,2024-02-15,2024-03-07T15:18:29.188000+00:00
4578,79974,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550403,2024-02-15,2024-03-07T15:18:29.188000+00:00
4579,79975,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550404,2024-02-15,2024-03-07T15:18:29.188000+00:00
4580,79976,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550405,2024-02-15,2024-03-07T15:18:29.188000+00:00


In [23]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FR55CP92140', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
84,534,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553311,2022-11-03,2024-03-18T17:18:52.660000+00:00
85,535,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553321,2022-11-03,2024-03-18T17:18:52.660000+00:00
86,536,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553331,2022-11-03,2024-03-18T17:18:52.660000+00:00
87,537,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553341,2022-11-03,2024-03-18T17:18:52.660000+00:00
88,538,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553351,2022-11-03,2024-03-18T17:18:52.660000+00:00
89,539,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553361,2022-11-03,2024-03-18T17:18:52.660000+00:00
90,540,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628411,2022-11-03,2024-03-18T17:18:52.660000+00:00
91,541,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628421,2022-11-03,2024-03-18T17:18:52.660000+00:00
92,542,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628431,2022-11-03,2024-03-18T17:18:52.660000+00:00
93,543,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628441,2022-11-03,2024-03-18T17:18:52.660000+00:00


### Repartition des defauts par operateur
Répartition pour les 5 opérateurs présentant le plus de défauts

In [24]:
print('PdC avec une incohérence : ')
for operateur in oper:
    print('\n', sum(itinerance_4.contact_operateur == operateur))
    print('Répartition des incohérences ', operateur, ' :')
    for cont in controles:
        #print(cont, sum((itinerance_4['contact_operateur'] == 'supervision-ev.france@totalenergies.com') & ~(itinerance_4[cont])))
        print('{:<50} {:>5}'.format(cont, sum((itinerance_4['contact_operateur'] == operateur) & ~(itinerance_4[cont]))))

PdC avec une incohérence : 

 2551
Répartition des incohérences  hello@powerdot.fr  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                   0
coordonneesXY - id_station_itinerance                 28
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                   28
implantation_station - id_station_itinerance           0
nbre_pdc - id_station_itinerance                    2551
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                       0
station_deux_roues - id_station_itinerance             0
adresse_station - coordonneesXY                        0

 595
Répartition des incohérences  supervision-ev.france@totalenergies.com  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance 

### Defauts operateur POWERDOT
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRPD1PCHAVDB' contient 28 points de recharge avec un nombre de points de rechage variable. Les identifiants des points de recharge et le nombre de PdC indiqué pour cet exemple montrent qu'il s'agit en réalité de 6 stations comportant chacune 3, 4, 8, 3, 8 et 2 PdC. Toutes les stations en défaut sont situées dans des parkings et concernent visiblement des stations implantées sur plusieurs étages et regroupées sous un même identifiant (on devrait donc a minima identifier une station par étage).

In [67]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'hello@powerdot.fr', relations][1000: 1500]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2703,52308,hello@powerdot.fr,Power Dot France,"[-0.6152723999999999, 43.1788568]",15 Bd de l'Aragon,FRPD1PGMVOLR,GAMM VERT OLORON,Parking privé à usage public,4,Accès libre,24/7,False,FRPD1EGMVOLRBBC200014,2024-03-13,2024-03-13T13:58:42.483000+00:00
2704,52309,hello@powerdot.fr,Power Dot France,"[0.0725598, 43.268522]",1 Rte de Bours,FRPD1PGMVTRB,GAMM VERT TARBES,Parking privé à usage public,1,Accès libre,24/7,False,FRPD1EGMVTRBALFS22011,2024-03-13,2024-03-13T13:58:42.483000+00:00
2705,52310,hello@powerdot.fr,Power Dot France,"[0.0725598, 43.268522]",1 Rte de Bours,FRPD1PGMVTRB,GAMM VERT TARBES,Parking privé à usage public,4,Accès libre,24/7,False,FRPD1EGMVTRBBBC200011,2024-03-13,2024-03-13T13:58:42.483000+00:00
2706,52311,hello@powerdot.fr,Power Dot France,"[0.0725598, 43.268522]",1 Rte de Bours,FRPD1PGMVTRB,GAMM VERT TARBES,Parking privé à usage public,4,Accès libre,24/7,False,FRPD1EGMVTRBBBC200012,2024-03-13,2024-03-13T13:58:42.483000+00:00
2707,52312,hello@powerdot.fr,Power Dot France,"[0.0725598, 43.268522]",1 Rte de Bours,FRPD1PGMVTRB,GAMM VERT TARBES,Parking privé à usage public,4,Accès libre,24/7,False,FRPD1EGMVTRBBBC200013,2024-03-13,2024-03-13T13:58:42.483000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3198,53159,hello@powerdot.fr,Power Dot France,"[1.246787573819, 43.964628154349]","13 Rue de la Mouscane, 82700 Montech, France",FRPD1PITMMTC,Intermarché Montech,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1EITMMTCKPS600017,2024-03-13,2024-03-13T13:58:42.483000+00:00
3199,53160,hello@powerdot.fr,Power Dot France,"[1.246787573819, 43.964628154349]","13 Rue de la Mouscane, 82700 Montech, France",FRPD1PITMMTC,Intermarché Montech,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1EITMMTCKPS600018,2024-03-13,2024-03-13T13:58:42.483000+00:00
3200,53161,hello@powerdot.fr,Power Dot France,"[1.153016, 45.05648]",les rives,FRPD1PITMMTG,Intermarché Super Montignac,Parking privé à usage public,1,Accès libre,24/7,False,FRPD1EITMMTGALF00011,2024-03-13,2024-03-13T13:58:42.483000+00:00
3201,53162,hello@powerdot.fr,Power Dot France,"[1.153016, 45.05648]",les rives,FRPD1PITMMTG,Intermarché Super Montignac,Parking privé à usage public,6,Accès libre,24/7,False,FRPD1EITMMTGKPS200011,2024-03-13,2024-03-13T13:58:42.483000+00:00


In [61]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRPD1PACCBEA', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
50845,50914,hello@powerdot.fr,Power Dot France,"[2.1128121025509, 49.410164265992]","1 Rue Pierre et Marie Curie, 60000 Beauvais, F...",FRPD1PACCBEA,HOTELF1 BEAUVAIS H2225,Parking privé à usage public,1,Accès libre,24/7,False,FRPD1EACCBEAALF0011,2024-03-13,2024-03-13T13:58:42.483000+00:00
50846,50915,hello@powerdot.fr,Power Dot France,"[2.1128121025509, 49.410164265992]","1 Rue Pierre et Marie Curie, 60000 Beauvais, F...",FRPD1PACCBEA,HOTELF1 BEAUVAIS H2225,Parking privé à usage public,4,Accès libre,24/7,False,FRPD1EACCBEAKP200011,2024-03-13,2024-03-13T13:58:42.483000+00:00
50847,50916,hello@powerdot.fr,Power Dot France,"[2.1128121025509, 49.410164265992]","1 Rue Pierre et Marie Curie, 60000 Beauvais, F...",FRPD1PACCBEA,HOTELF1 BEAUVAIS H2225,Parking privé à usage public,4,Accès libre,24/7,False,FRPD1EACCBEAKP200012,2024-03-13,2024-03-13T13:58:42.483000+00:00
50848,50917,hello@powerdot.fr,Power Dot France,"[2.1128121025509, 49.410164265992]","1 Rue Pierre et Marie Curie, 60000 Beauvais, F...",FRPD1PACCBEA,HOTELF1 BEAUVAIS H2225,Parking privé à usage public,4,Accès libre,24/7,False,FRPD1EACCBEAKP200013,2024-03-13,2024-03-13T13:58:42.483000+00:00
50849,50918,hello@powerdot.fr,Power Dot France,"[2.1128121025509, 49.410164265992]","1 Rue Pierre et Marie Curie, 60000 Beauvais, F...",FRPD1PACCBEA,HOTELF1 BEAUVAIS H2225,Parking privé à usage public,4,Accès libre,24/7,False,FRPD1EACCBEAKP200014,2024-03-13,2024-03-13T13:58:42.483000+00:00


In [60]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRPD1PCHAVDB', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
51331,51400,hello@powerdot.fr,Power Dot France,"[5.9483640919382, 49.254201711321]","2 Rue Olympe de Gouges, 54150 Val de Briey, Fr...",FRPD1PCHAVDB,Chaussea Val de Briey,Parking privé à usage public,2,Accès libre,24/7,False,FRPD1ECHAVDBHUB1ALF0021,2024-03-13,2024-03-13T13:58:42.483000+00:00
51332,51401,hello@powerdot.fr,Power Dot France,"[5.9483640919382, 49.254201711321]","2 Rue Olympe de Gouges, 54150 Val de Briey, Fr...",FRPD1PCHAVDB,Chaussea Val de Briey,Parking privé à usage public,2,Accès libre,24/7,False,FRPD1ECHAVDBHUB1ALF0022,2024-03-13,2024-03-13T13:58:42.483000+00:00
51333,51402,hello@powerdot.fr,Power Dot France,"[5.9483640919382, 49.254201711321]","2 Rue Olympe de Gouges, 54150 Val de Briey, Fr...",FRPD1PCHAVDB,Chaussea Val de Briey,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ECHAVDBHUB1KP0011,2024-03-13,2024-03-13T13:58:42.483000+00:00
51334,51403,hello@powerdot.fr,Power Dot France,"[5.9483640919382, 49.254201711321]","2 Rue Olympe de Gouges, 54150 Val de Briey, Fr...",FRPD1PCHAVDB,Chaussea Val de Briey,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ECHAVDBHUB1KP0012,2024-03-13,2024-03-13T13:58:42.483000+00:00
51335,51404,hello@powerdot.fr,Power Dot France,"[5.9483640919382, 49.254201711321]","2 Rue Olympe de Gouges, 54150 Val de Briey, Fr...",FRPD1PCHAVDB,Chaussea Val de Briey,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ECHAVDBHUB1KP0013,2024-03-13,2024-03-13T13:58:42.483000+00:00
51336,51405,hello@powerdot.fr,Power Dot France,"[5.9483640919382, 49.254201711321]","2 Rue Olympe de Gouges, 54150 Val de Briey, Fr...",FRPD1PCHAVDB,Chaussea Val de Briey,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ECHAVDBHUB1KP0014,2024-03-13,2024-03-13T13:58:42.483000+00:00
51337,51406,hello@powerdot.fr,Power Dot France,"[5.9483640919382, 49.254201711321]","2 Rue Olympe de Gouges, 54150 Val de Briey, Fr...",FRPD1PCHAVDB,Chaussea Val de Briey,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ECHAVDBHUB1KP0015,2024-03-13,2024-03-13T13:58:42.483000+00:00
51338,51407,hello@powerdot.fr,Power Dot France,"[5.9483640919382, 49.254201711321]","2 Rue Olympe de Gouges, 54150 Val de Briey, Fr...",FRPD1PCHAVDB,Chaussea Val de Briey,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ECHAVDBHUB1KP0016,2024-03-13,2024-03-13T13:58:42.483000+00:00
51339,51408,hello@powerdot.fr,Power Dot France,"[5.9483640919382, 49.254201711321]","2 Rue Olympe de Gouges, 54150 Val de Briey, Fr...",FRPD1PCHAVDB,Chaussea Val de Briey,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ECHAVDBHUB1KP0017,2024-03-13,2024-03-13T13:58:42.483000+00:00
51340,51409,hello@powerdot.fr,Power Dot France,"[5.9483640919382, 49.254201711321]","2 Rue Olympe de Gouges, 54150 Val de Briey, Fr...",FRPD1PCHAVDB,Chaussea Val de Briey,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ECHAVDBHUB1KP0018,2024-03-13,2024-03-13T13:58:42.483000+00:00


### Defauts operateur ALIZECHARGE
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRADPP91479018' contient 82 points de recharge avec plusieurs adresses et un nombre de points de rechage variable. Cet exemple concerne des stations implantées sur plusieurs étages d'un parking (on devrait donc a minima identifier une station par étage).

In [25]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
271,1813,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE914790181,2024-02-15,2024-03-07T15:18:29.188000+00:00
272,1814,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901810,2024-02-15,2024-03-07T15:18:29.188000+00:00
273,1815,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901811,2024-02-15,2024-03-07T15:18:29.188000+00:00
274,1816,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901812,2024-02-15,2024-03-07T15:18:29.188000+00:00
275,1817,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901813,2024-02-15,2024-03-07T15:18:29.188000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4576,79972,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550401,2024-02-15,2024-03-07T15:18:29.188000+00:00
4577,79973,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550402,2024-02-15,2024-03-07T15:18:29.188000+00:00
4578,79974,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550403,2024-02-15,2024-03-07T15:18:29.188000+00:00
4579,79975,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550404,2024-02-15,2024-03-07T15:18:29.188000+00:00


In [26]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRADPP91479018', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1813,1813,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE914790181,2024-02-15,2024-03-07T15:18:29.188000+00:00
1814,1814,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901810,2024-02-15,2024-03-07T15:18:29.188000+00:00
1815,1815,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901811,2024-02-15,2024-03-07T15:18:29.188000+00:00
1816,1816,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901812,2024-02-15,2024-03-07T15:18:29.188000+00:00
1817,1817,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901813,2024-02-15,2024-03-07T15:18:29.188000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,1890,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE914790188,2024-02-15,2024-03-07T15:18:29.188000+00:00
1891,1891,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-Poste,FRADPP91479018,ORY - Parking P3 SILO - Niveau 6,Voirie,18,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901880,2024-02-15,2024-03-07T15:18:29.188000+00:00
1892,1892,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-Poste,FRADPP91479018,ORY - Parking P3 SILO - Niveau 6,Voirie,18,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901881,2024-02-15,2024-03-07T15:18:29.188000+00:00
1893,1893,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-Poste,FRADPP91479018,ORY - Parking P3 SILO - Niveau 6,Voirie,18,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901882,2024-02-15,2024-03-07T15:18:29.188000+00:00


In [27]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRTLSP31555040', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
79175,79972,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550401,2024-02-15,2024-03-07T15:18:29.188000+00:00
79176,79973,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550402,2024-02-15,2024-03-07T15:18:29.188000+00:00
79177,79974,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550403,2024-02-15,2024-03-07T15:18:29.188000+00:00
79178,79975,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550404,2024-02-15,2024-03-07T15:18:29.188000+00:00
79179,79976,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550405,2024-02-15,2024-03-07T15:18:29.188000+00:00


### Defauts operateur IZIVIA
Les défauts au nombre de 280 sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Izivia en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance, ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur un parking [-0.530598, 47.398424], on trouve 17 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué entre 1 et 3.

In [28]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'sav@izivia.com', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
545,11093,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24711,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24711,2024-03-26,2024-03-27T00:29:18.671000+00:00
546,11094,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24712,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24712,2024-03-26,2024-03-27T00:29:18.671000+00:00
547,11095,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24721,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24721,2024-03-26,2024-03-27T00:29:18.671000+00:00
548,11096,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24722,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24722,2024-03-26,2024-03-27T00:29:18.671000+00:00
549,11097,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24723,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24723,2024-03-26,2024-03-27T00:29:18.671000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4624,82412,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1622,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1622,2024-03-26,2024-03-27T00:29:15.919000+00:00
4625,82413,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1631,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1631,2024-03-26,2024-03-27T00:29:15.919000+00:00
4626,82414,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1632,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1632,2024-03-26,2024-03-27T00:29:15.919000+00:00
4627,82415,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL AEROVILLE 95700 ROISSY-EN-FR...,FRURWPUNIB1711,AEROVILLE - TOKYO,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1711,2024-03-26,2024-03-27T00:29:15.919000+00:00


In [29]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-0.530598, 47.398424]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
545,11093,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24711,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24711,2024-03-26,2024-03-27T00:29:18.671000+00:00
546,11094,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24712,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24712,2024-03-26,2024-03-27T00:29:18.671000+00:00
547,11095,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24721,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24721,2024-03-26,2024-03-27T00:29:18.671000+00:00
548,11096,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24722,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24722,2024-03-26,2024-03-27T00:29:18.671000+00:00
549,11097,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24723,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24723,2024-03-26,2024-03-27T00:29:18.671000+00:00
1674,44643,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372101,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372101,2024-03-26,2024-03-26T15:33:42.643000+00:00
1675,44644,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR37211,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR37211,2024-03-26,2024-03-26T15:33:42.643000+00:00
1676,44645,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372111,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372111,2024-03-26,2024-03-26T15:33:42.643000+00:00
1677,44646,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372121,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372121,2024-03-26,2024-03-26T15:33:42.643000+00:00
1678,44647,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR37221,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR37221,2024-03-26,2024-03-26T15:33:42.643000+00:00


### Defauts operateur IONITY
Les défauts (480 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Ionity en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance ainsi qu'un ancien pdc de regroupement par station avec un nom d'enseigne différent (IONITY GMBH), ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur l'aire de Mornas [4.732409, 44.194832], on trouve 26 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué de 23 pour chaque station.

In [30]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
820,32755,info@ionity.eu,IONITY GMBH,"[0.366838, 46.701436]","Aire de Poitou-Charentes, A10, 79180 Vouillé",FRIONE124800,IONITY Poitou Charentes,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1248,2023-03-21,2024-01-19T07:47:22.735000+00:00
821,32756,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
822,32757,info@ionity.eu,IONITY GMBH,"[1.578674, 48.467261]","Aire De Chartres Bois,A11 - 28300 Gasville-Oiséme",FRIONE126500,IONITY Chartres Bois Paris,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1265,2023-03-21,2024-01-19T07:47:22.735000+00:00
823,32758,info@ionity.eu,IONITY GMBH,"[3.918671, 49.242974]","Aire de Vrigny,A4-51390 Vrigny",FRIONE401700,IONITY Vrigny,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4017,2023-03-21,2024-01-19T07:47:22.735000+00:00
824,32761,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2024-01-19T07:47:22.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,33819,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440353,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440353,2024-03-26,2024-03-27T00:00:15.372000+00:00
1247,33820,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440371,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440371,2024-03-26,2024-03-27T00:00:15.372000+00:00
1248,33821,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440372,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440372,2024-03-26,2024-03-27T00:00:15.372000+00:00
1249,33822,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440373,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440373,2024-03-26,2024-03-27T00:00:15.372000+00:00


In [31]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
821,32756,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
869,32899,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126401,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126401,2024-03-26,2024-03-27T00:00:15.372000+00:00
870,32900,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126402,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126402,2024-03-26,2024-03-27T00:00:15.372000+00:00
871,32901,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126404,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126404,2024-03-26,2024-03-27T00:00:15.372000+00:00
872,32902,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126405,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126405,2024-03-26,2024-03-27T00:00:15.372000+00:00
873,32903,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126451,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126451,2024-03-26,2024-03-27T00:00:15.372000+00:00
874,32904,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126452,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126452,2024-03-26,2024-03-27T00:00:15.372000+00:00
875,32905,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126453,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126453,2024-03-26,2024-03-27T00:00:15.372000+00:00


In [32]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[4.732409, 44.194832]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
867,32862,info@ionity.eu,IONITY GMBH,"[4.732409, 44.194832]","Aire de Mornas Village , A7, 84550 Mornas",FRIONE440300,IONITY Mornas Village,Station dédiée à la recharge rapide,16,Accès libre,24/7,FALSE,FRIONE4403,2023-03-21,2024-01-19T07:47:22.735000+00:00
1228,33801,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440301,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440301,2024-03-26,2024-03-27T00:00:15.372000+00:00
1229,33802,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440302,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440302,2024-03-26,2024-03-27T00:00:15.372000+00:00
1230,33803,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440303,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440303,2024-03-26,2024-03-27T00:00:15.372000+00:00
1231,33804,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440304,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440304,2024-03-26,2024-03-27T00:00:15.372000+00:00
1232,33805,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440305,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440305,2024-03-26,2024-03-27T00:00:15.372000+00:00
1233,33806,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440306,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440306,2024-03-26,2024-03-27T00:00:15.372000+00:00
1234,33807,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440307,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440307,2024-03-26,2024-03-27T00:00:15.372000+00:00
1235,33808,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440308,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440308,2024-03-26,2024-03-27T00:00:15.372000+00:00
1236,33809,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440309,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440309,2024-03-26,2024-03-27T00:00:15.372000+00:00


In [33]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
820,32755,info@ionity.eu,IONITY GMBH,"[0.366838, 46.701436]","Aire de Poitou-Charentes, A10, 79180 Vouillé",FRIONE124800,IONITY Poitou Charentes,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1248,2023-03-21,2024-01-19T07:47:22.735000+00:00
821,32756,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
822,32757,info@ionity.eu,IONITY GMBH,"[1.578674, 48.467261]","Aire De Chartres Bois,A11 - 28300 Gasville-Oiséme",FRIONE126500,IONITY Chartres Bois Paris,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1265,2023-03-21,2024-01-19T07:47:22.735000+00:00
823,32758,info@ionity.eu,IONITY GMBH,"[3.918671, 49.242974]","Aire de Vrigny,A4-51390 Vrigny",FRIONE401700,IONITY Vrigny,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4017,2023-03-21,2024-01-19T07:47:22.735000+00:00
824,32761,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2024-01-19T07:47:22.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,33819,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440353,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440353,2024-03-26,2024-03-27T00:00:15.372000+00:00
1247,33820,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440371,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440371,2024-03-26,2024-03-27T00:00:15.372000+00:00
1248,33821,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440372,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440372,2024-03-26,2024-03-27T00:00:15.372000+00:00
1249,33822,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440373,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440373,2024-03-26,2024-03-27T00:00:15.372000+00:00


In [34]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.366838, 46.701436]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
820,32755,info@ionity.eu,IONITY GMBH,"[0.366838, 46.701436]","Aire de Poitou-Charentes, A10, 79180 Vouillé",FRIONE124800,IONITY Poitou Charentes,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1248,2023-03-21,2024-01-19T07:47:22.735000+00:00
1041,33230,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409401,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409401,2024-03-26,2024-03-27T00:00:15.372000+00:00
1042,33231,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409402,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409402,2024-03-26,2024-03-27T00:00:15.372000+00:00
1043,33232,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409403,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409403,2024-03-26,2024-03-27T00:00:15.372000+00:00
1044,33233,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409404,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409404,2024-03-26,2024-03-27T00:00:15.372000+00:00
1045,33234,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409451,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409451,2024-03-26,2024-03-27T00:00:15.372000+00:00
1046,33235,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409452,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409452,2024-03-26,2024-03-27T00:00:15.372000+00:00
1047,33236,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409453,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409453,2024-03-26,2024-03-27T00:00:15.372000+00:00


In [35]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.523685, 48.9908]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4609,82394,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1311,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1311,2024-03-26,2024-03-27T00:29:15.919000+00:00
4610,82395,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1312,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1312,2024-03-26,2024-03-27T00:29:15.919000+00:00
4611,82396,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1321,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1321,2024-03-26,2024-03-27T00:29:15.919000+00:00
4612,82397,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1322,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1322,2024-03-26,2024-03-27T00:29:15.919000+00:00
4613,82399,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1411,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1411,2024-03-26,2024-03-27T00:29:15.919000+00:00
4614,82400,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1412,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1412,2024-03-26,2024-03-27T00:29:15.919000+00:00
4615,82401,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1421,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1421,2024-03-26,2024-03-27T00:29:15.919000+00:00
4616,82402,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1422,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1422,2024-03-26,2024-03-27T00:29:15.919000+00:00
4617,82404,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1511,AEROVILLE - NORDIC CHIC,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1511,2024-03-26,2024-03-27T00:29:15.919000+00:00
4618,82405,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1512,AEROVILLE - NORDIC CHIC,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1512,2024-03-26,2024-03-27T00:29:15.919000+00:00


### Cohérence implantation_station - id_station
- 107 pdc sont liés à une erreur de choix d'implantation (couplé à d'autres erreurs)

In [36]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
42,340,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300611,2022-11-03,2024-03-18T17:18:52.660000+00:00
43,341,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Voirie,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300621,2022-11-03,2024-03-18T17:18:52.660000+00:00
44,342,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300631,2022-11-03,2024-03-18T17:18:52.660000+00:00
45,343,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300641,2022-11-03,2024-03-18T17:18:52.660000+00:00
528,7501,support@alizecharge.fr,Métropole Rouen Normandie,"[0.872578, 49.479934]",Avenue du Président Coty 76480 DUCLAIR,FRC01P76222001,DUCLAIR - Avenue du Président Coty,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRC01E762220011,2024-02-15,2024-03-07T15:18:29.188000+00:00
529,7502,support@alizecharge.fr,Métropole Rouen Normandie,"[0.872578, 49.479934]",Avenue du Président Coty 76480 DUCLAIR,FRC01P76222001,DUCLAIR - Avenue du Président Coty,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRC01E762220012,2024-02-15,2024-03-07T15:18:29.188000+00:00
530,7503,support@alizecharge.fr,Métropole Rouen Normandie,"[0.87699, 49.48461]",52 Chemin du Catel 76480 DUCLAIR,FRC01P76222001,DUCLAIR - Parking Chemin du Catel,Voirie,2,Accès libre,Mo-Su 00:00-23:59,False,FRC01E762220013,2024-02-15,2024-03-07T15:18:29.188000+00:00
531,7504,support@alizecharge.fr,Métropole Rouen Normandie,"[0.87699, 49.48461]",52 Chemin du Catel 76480 DUCLAIR,FRC01P76222001,DUCLAIR - Parking Chemin du Catel,Voirie,2,Accès libre,Mo-Su 00:00-23:59,False,FRC01E762220014,2024-02-15,2024-03-07T15:18:29.188000+00:00
815,32747,support@iecharge.io,Vezin-le-Coquet,"[-1.7547021, 48.1252516]","4001 Rue de la Belle-Epine, 35132 Vezin-le-Coquet",FRIENE35353A,Vezin-le-Coquet,Station dédiée à la recharge rapide,5,Accès réservé,24/7,false,FRIENE35353A1,2024-03-26,2024-03-27T00:00:15.372000+00:00
816,32748,support@iecharge.io,Vezin-le-Coquet,"[-1.7547021, 48.1252516]","4001 Rue de la Belle-Epine, 35132 Vezin-le-Coquet",FRIENE35353A,Vezin-le-Coquet,Station dédiée à la recharge rapide,5,Accès réservé,24/7,false,FRIENE35353A2,2024-03-26,2024-03-27T00:00:15.372000+00:00


### Cohérence nom_station - id_station
- 210 pdc sont associés à une station avec un nom non cohérent

In [37]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
46,348,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643461,2022-11-03,2024-03-18T17:18:52.660000+00:00
47,349,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643471,2022-11-03,2024-03-18T17:18:52.660000+00:00
48,350,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684251,2022-11-03,2024-03-18T17:18:52.660000+00:00
49,351,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684261,2022-11-03,2024-03-18T17:18:52.660000+00:00
50,352,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.768425, 43.958425]",PARKING ELEPHANT BLEU - LES ANGLES,FR55CP30133,PARKING ELEPHANT BLEU - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334396228947525161,2022-11-03,2024-03-18T17:18:52.660000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4576,79972,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550401,2024-02-15,2024-03-07T15:18:29.188000+00:00
4577,79973,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550402,2024-02-15,2024-03-07T15:18:29.188000+00:00
4578,79974,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550403,2024-02-15,2024-03-07T15:18:29.188000+00:00
4579,79975,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550404,2024-02-15,2024-03-07T15:18:29.188000+00:00


### Coherence adresse - coordonnees
- 1807 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [38]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
18,138,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","400 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:00,false,FR3R3E10000849681,2023-12-22,2024-01-29T07:15:04.845000+00:00
19,139,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","401 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:01,false,FR3R3E10000849682,2023-12-22,2024-01-29T07:15:04.845000+00:00
20,140,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",708 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849691,2023-12-22,2024-01-29T07:15:04.845000+00:00
21,141,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",709 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849692,2023-12-22,2024-01-29T07:15:04.845000+00:00
22,158,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",713 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849851,2023-12-22,2024-01-29T07:15:04.845000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5122,87253,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2024-01-19T07:46:24.141000+00:00
5123,87262,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
5124,87263,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
5125,87264,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [39]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[5.2592028, 46.1746523]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
5123,87262,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
5124,87263,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
5125,87264,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00
5126,87265,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [40]:
itinerance_4.loc[itinerance_4.adresse_station == 'RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
5123,87262,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
5125,87264,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [41]:
itinerance_4.loc[itinerance_4.adresse_station == ' RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
5124,87263,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
5126,87265,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-01-19T07:46:23.908000+00:00


- exemple : Stations Ouest charge (double saisie par deux opérateurs)

In [42]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-3.577855, 48.771084]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Cohérence station - coordonnées
- 323 pdc sont associés à des stations avec plusieurs coordonnées (dont 127 Freshmile)

In [43]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][300:315]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4084,54560,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,3,Accès libre,24/7,False,FRPD1ESYULGNEKO60012,2024-03-13,2024-03-13T13:58:42.483000+00:00
4085,54561,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,3,Accès libre,24/7,False,FRPD1ESYULGNEKO60013,2024-03-13,2024-03-13T13:58:42.483000+00:00
4086,54562,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,3,Accès libre,24/7,False,FRPD1ESYULGNEKO60021,2024-03-13,2024-03-13T13:58:42.483000+00:00
4087,54563,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,3,Accès libre,24/7,False,FRPD1ESYULGNEKO60022,2024-03-13,2024-03-13T13:58:42.483000+00:00
4088,54564,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,3,Accès libre,24/7,False,FRPD1ESYULGNEKO60023,2024-03-13,2024-03-13T13:58:42.483000+00:00
4096,54576,hello@powerdot.fr,Power Dot France,"[1.396961, 45.436744]",Rue du Huit Mai 1945,FRPD1PSYULUB,Super U Lubersac,Parking privé à usage public,2,Accès libre,24/7,False,FRPD1ESYULUBBBP150011,2024-03-13,2024-03-13T13:58:42.483000+00:00
4097,54577,hello@powerdot.fr,Power Dot France,"[1.396961, 45.436744]",Rue du Huit Mai 1945,FRPD1PSYULUB,Super U Lubersac,Parking privé à usage public,2,Accès libre,24/7,False,FRPD1ESYULUBBBP150012,2024-03-13,2024-03-13T13:58:42.483000+00:00
4098,54578,hello@powerdot.fr,Power Dot France,"[1.421600322937, 45.449529333113]","CCXC+MH, 19210 Lubersac, France",FRPD1PSYULUB,Station-service Super U Touvent,Parking privé à usage public,3,Accès libre,24/7,False,FRPD1ESYULUBKPS50011,2024-03-13,2024-03-13T13:58:42.483000+00:00
4099,54579,hello@powerdot.fr,Power Dot France,"[1.421600322937, 45.449529333113]","CCXC+MH, 19210 Lubersac, France",FRPD1PSYULUB,Station-service Super U Touvent,Parking privé à usage public,3,Accès libre,24/7,False,FRPD1ESYULUBKPS50012,2024-03-13,2024-03-13T13:58:42.483000+00:00
4100,54580,hello@powerdot.fr,Power Dot France,"[1.421600322937, 45.449529333113]","CCXC+MH, 19210 Lubersac, France",FRPD1PSYULUB,Station-service Super U Touvent,Parking privé à usage public,3,Accès libre,24/7,False,FRPD1ESYULUBKPS50013,2024-03-13,2024-03-13T13:58:42.483000+00:00


In [44]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRLE2PYSCJHWOOBD', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [45]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRFR1PEHNKRRPQNR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [46]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,18,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2024-01-19T07:46:04.014000+00:00
1,19,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2024-01-19T07:46:04.014000+00:00
2,20,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2024-01-19T07:46:04.014000+00:00
3,21,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2024-01-19T07:46:04.014000+00:00
5,23,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2024-01-19T07:46:04.014000+00:00
6,24,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2024-01-19T07:46:04.014000+00:00
7,25,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2024-01-19T07:46:04.014000+00:00
8,26,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2024-01-19T07:46:04.014000+00:00
9,27,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2024-01-19T07:46:04.014000+00:00
10,28,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2024-01-19T07:46:04.014000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [47]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
52,359,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2024-03-18T17:18:52.660000+00:00
53,360,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2024-03-18T17:18:52.660000+00:00
54,361,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2024-03-18T17:18:52.660000+00:00
55,362,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2024-03-18T17:18:52.660000+00:00
56,363,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2024-03-18T17:18:52.660000+00:00
57,364,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2024-03-18T17:18:52.660000+00:00


- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [48]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.08745032, 48.40887156]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [49]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [50]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4127,68271,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 1 64000 PAU,FRS64PMB649711,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649711,2024-03-26,2024-03-27T00:29:12.195000+00:00
4128,68272,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 1 64000 PAU,FRS64PMB649712,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649712,2024-03-26,2024-03-27T00:29:12.195000+00:00
4129,68273,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 2 64000 PAU,FRS64PMB649811,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649811,2024-03-26,2024-03-27T00:29:12.195000+00:00
4130,68274,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 2 64000 PAU,FRS64PMB649812,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649812,2024-03-26,2024-03-27T00:29:12.195000+00:00


### Incoherence nbre_pdc
- 583 pdc ont un champ 'nbre_pdc' mal documenté

In [51]:
itinerance_4.loc[~itinerance_4['nbre_pdc - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
603,19252,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,6,Accès libre,24/7,false,FRETIE59368A11,2024-03-13,2024-03-14T07:13:58.423000+00:00
604,19253,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,6,Accès libre,24/7,false,FRETIE59368A12,2024-03-13,2024-03-14T07:13:58.423000+00:00
605,19254,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,6,Accès libre,24/7,false,FRETIE59368A13,2024-03-13,2024-03-14T07:13:58.423000+00:00
606,19255,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,6,Accès libre,24/7,false,FRETIE59368A14,2024-03-13,2024-03-14T07:13:58.423000+00:00
607,19256,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,6,Accès libre,24/7,false,FRETIE59368A15,2024-03-13,2024-03-14T07:13:58.423000+00:00
608,19257,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,6,Accès libre,24/7,false,FRETIE59368A16,2024-03-13,2024-03-14T07:13:58.423000+00:00
609,19258,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,11,Accès libre,24/7,false,FRETIE59368A21,2024-03-13,2024-03-14T07:13:58.423000+00:00
610,19259,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,11,Accès libre,24/7,false,FRETIE59368A210,2024-03-13,2024-03-14T07:13:58.423000+00:00
611,19260,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,11,Accès libre,24/7,false,FRETIE59368A211,2024-03-13,2024-03-14T07:13:58.423000+00:00
612,19261,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,11,Accès libre,24/7,false,FRETIE59368A22,2024-03-13,2024-03-14T07:13:58.423000+00:00


### coherence station - enseigne
- 5 stations ont plusieurs noms d'enseigne

In [52]:
itinerance_4.loc[~itinerance_4['nom_enseigne - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
641,22138,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
642,22139,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
643,22140,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2024-01-19T07:50:45.527000+00:00
1298,38857,contact@rirodo.fr,LAST MILES SOLUTIONS,"[5.39, 43.27]",20 Rue Negresko,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRLMSE1897228,2023-09-06,2024-01-19T07:49:16.739000+00:00
4256,55568,contact@rirodo.fr,MONTA,"[5.39, 43.27]",20 Rue NEGRESKO,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRRIRE2804856,2023-09-06,2024-01-19T07:49:16.739000+00:00


In [53]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRFR1EAYRU', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
21386,21408,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-01-19T07:49:57.766000+00:00
21388,21410,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-01-19T07:49:57.766000+00:00
21390,21412,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU3,2022-08-25,2024-01-19T07:49:57.766000+00:00
22116,22138,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
22117,22139,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
22118,22140,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2024-01-19T07:50:45.527000+00:00


In [54]:
itinerance_init.loc[itinerance_init.nom_enseigne == 'RME Loos (FR-FR1)', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
21386,21408,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-01-19T07:49:57.766000+00:00
21388,21410,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-01-19T07:49:57.766000+00:00
22116,22138,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
22117,22139,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
25227,25249,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS1,2022-10-04,2024-01-19T07:49:31.469000+00:00
25229,25251,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS2,2022-10-04,2024-01-19T07:49:31.469000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire (15)

In [55]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues (583)

In [56]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
83577,84374,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17161,2023-07-06,2024-01-19T07:47:27.885000+00:00
83578,84375,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17162,2023-07-06,2024-01-19T07:47:27.885000+00:00
83579,84376,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17163,2023-07-06,2024-01-19T07:47:27.885000+00:00
83580,84377,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17164,2023-07-06,2024-01-19T07:47:27.885000+00:00
83581,84378,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17165,2023-07-06,2024-01-19T07:47:27.885000+00:00
83582,84379,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17166,2023-07-06,2024-01-19T07:47:27.885000+00:00
83583,84380,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17167,2023-07-06,2024-01-19T07:47:27.885000+00:00
